<a href="https://colab.research.google.com/github/mownikakolli/IIIT_FMML_Lab_Assignment/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1 **answer**


The percentage of the validation set in a machine learning experiment can have a significant impact on model training and evaluation. Let's explore how increasing and reducing the percentage of the validation set can affect validation accuracy:

Increasing the Percentage of the Validation Set:

Pros:

More data for validation can lead to a more reliable estimate of model performance. A larger validation set provides a better representation of the model's generalization capabilities.
It can help in identifying overfitting more accurately. A larger validation set is more likely to capture the model's tendency to overfit the training data.
Cons:

You have less data available for training, which can lead to longer training times or, in some cases, difficulty in training a complex model effectively.
If you have a limited dataset, allocating a larger portion to the validation set may reduce the amount of data the model can learn from, potentially leading to underfitting.
Impact on Validation Accuracy:

Initially, as you increase the percentage of the validation set, validation accuracy may improve because you have a better estimate of generalization performance.
However, if you allocate too much data to the validation set, your model might not have enough data to train effectively, and validation accuracy may start to degrade.
Reducing the Percentage of the Validation Set:

Pros:

More data for training can help the model learn more effectively, especially if you have a limited dataset.
Training times may be shorter, as you have a larger portion of data available for training.
Cons:

The estimate of model performance on unseen data may be less reliable because the validation set is smaller and might not represent the data distribution well.
It can be harder to detect overfitting since the validation set may not be large enough to capture subtle overfitting issues.
Impact on Validation Accuracy:

Initially, as you reduce the percentage of the validation set, training accuracy may improve because the model has more data to learn from.
However, if you allocate too little data to the validation set, your estimate of generalization performance may become less accurate, and you might not detect overfitting issues until you apply the model to new, unseen data.
In practice, the choice of the percentage of the validation set depends on the specific problem, dataset size, and computational resources available. It's often recommended to split the data into training, validation, and test sets while ensuring that the validation set is large enough to provide a reliable estimate of model performance but not so large that it hinders effective model training. Cross-validation techniques can also be employed to mitigate the impact of limited validation data.

2 answer
The size of the training and validation sets can indeed affect how well we can predict the accuracy on the test set using the validation set. Here's how:

Large Training Set, Small Validation Set:

When you have a large training set and a small validation set, you might have a highly capable model because it has access to a lot of training data. However, the small validation set might not provide a reliable estimate of the model's performance on unseen data.

Predicting accuracy on the test set based on the validation set in this scenario can be less reliable. A model that performs well on a small validation set might still perform poorly on the larger, more diverse test set.

Small Training Set, Large Validation Set:

With a small training set and a large validation set, you have a more reliable estimate of the model's performance on your specific validation data. However, the model might not generalize well to the test set if it hasn't seen enough diverse examples during training.

Predicting accuracy on the test set based on the validation set in this scenario might be more reliable for the validation set itself but may still overestimate the model's true performance on unseen data.

Balanced Training and Validation Set Sizes:

Ideally, you want a balanced allocation of data between the training and validation sets. This allows the model to learn from a reasonable amount of data while having a representative validation set for performance estimation.

In this case, predicting accuracy on the test set based on the validation set tends to be more reliable. If the validation set is a good representation of the test set in terms of data distribution and diversity, the validation performance is a reasonable indicator of how the model will perform on unseen data.

Cross-Validation:

To further improve the reliability of predicting test set accuracy, you can use techniques like k-fold cross-validation. In k-fold cross-validation, the dataset is divided into k subsets, and the model is trained and validated k times, with each subset serving as the validation set once.

This approach helps in obtaining a more robust estimate of the model's performance because it reduces the dependency on a single validation set. The average validation performance across k folds is a better indicator of how the model is likely to perform on unseen data.



**3 answer**


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**1 answer**


Yes, averaging the validation accuracy across multiple splits, such as in k-fold cross-validation, can provide more consistent and reliable results compared to relying on a single validation split. Here's why averaging validation accuracy is beneficial:

Reduces Variance: Averaging across multiple splits helps to reduce the variance in the performance estimate. With a single validation split, the performance estimate can be highly dependent on the particular data points chosen for validation. In contrast, cross-validation repeatedly partitions the data into different training and validation sets, providing a more stable estimate of model performance.

Better Generalization: By using different subsets of data for validation in each fold, cross-validation helps assess how well the model generalizes to various portions of the dataset. This is especially valuable when the dataset is limited or contains variations in data distribution.

More Robust Model Evaluation: Cross-validation allows you to obtain multiple performance metrics, each corresponding to a different validation fold. You can calculate various statistics, such as mean, standard deviation, or confidence intervals, to gain a better understanding of the model's expected performance and its variability.

Reduced Bias: Averaging results from multiple folds can help mitigate any potential bias introduced by a specific validation split. It ensures that the model's performance is evaluated across various subsets of the data, reducing the risk of obtaining overly optimistic or pessimistic estimates.

Improved Hyperparameter Tuning: When tuning hyperparameters or comparing different models, cross-validation provides a more accurate basis for model selection. It helps you choose the model that performs consistently well across multiple validation folds rather than one that performs well on a single split by chance.

Overall, cross-validation, by averaging validation accuracy across multiple splits, is a standard technique in machine learning for robust model evaluation. It provides a more dependable estimate of a model's performance and allows you to make more informed decisions regarding model selection, hyperparameter tuning, and assessing model generalization.







**2 answer**

Cross-validation, specifically k-fold cross-validation, provides a more accurate estimate of test accuracy compared to a single validation split when you don't have access to a separate test dataset. However, it's important to understand the terminology and context:

1. **Validation Accuracy vs. Test Accuracy**:

   - **Validation Accuracy**: This is an estimate of how well your model is performing on data that it has not seen during training but is still part of your dataset. It helps you tune hyperparameters and assess the model's performance during development.

   - **Test Accuracy**: This is an estimate of how well your model is expected to perform on completely unseen, real-world data. Test accuracy is what you're ultimately interested in, as it reflects the model's ability to generalize to new, unknown examples.

2. **Using Cross-Validation for Test Accuracy Estimation**:

   - When you don't have a separate, dedicated test dataset, you can use k-fold cross-validation to get a more accurate estimate of how your model is likely to perform on unseen data.

   - In k-fold cross-validation, you partition your dataset into k subsets or folds. You then train and validate your model k times, using each fold as the validation set once while the remaining k-1 folds are used for training.

   - After each iteration, you can calculate a validation accuracy. When you finish all k iterations, you can average these validation accuracies to obtain a more robust estimate of your model's performance.

   - While the averaged validation accuracy is not precisely the same as test accuracy on truly unseen data, it is a better estimate than a single validation split. It provides a more reliable indicator of how well your model is likely to generalize.

3. **Keep in Mind**:

   - Cross-validation can give you a more accurate estimate of how your model will perform on unseen data within the context of your dataset. However, it cannot replace a true test dataset, which represents data from the real world that your model has never encountered before.

   - To obtain a truly unbiased estimate of test accuracy, it is recommended, whenever possible, to set aside a separate test dataset that you do not use during model development or hyperparameter tuning. This ensures that your test accuracy estimate is as close as possible to the model's performance in a real-world scenario.

In summary, while k-fold cross-validation provides a more accurate estimate of how your model is likely to perform on unseen data within the dataset you have, it is not a substitute for a dedicated test dataset when you need to evaluate your model's performance in real-world scenarios. Test datasets are crucial for ensuring the reliability of your model's performance estimates.

**3 answer**

In the context of k-fold cross-validation, the number of iterations, or folds (k), can have an impact on the estimate of model performance. Generally, increasing the number of iterations can lead to a more reliable estimate of model performance, up to a certain point. Here's how the number of iterations affects the estimate:

1. **Effect of Increasing Iterations**:

   - **Pros**:
     - **Stability**: With a larger number of iterations (higher k), the estimate of model performance becomes more stable and less dependent on the specific data split. This reduces the impact of randomness in the selection of training and validation sets.

     - **Reduced Bias**: More iterations help in reducing bias in the performance estimate. It ensures that the model is evaluated on a more diverse set of validation data, which can provide a more balanced view of its generalization ability.

     - **Better Confidence Intervals**: Averaging results from more iterations allows for more accurate calculation of confidence intervals, helping you understand the range within which the true model performance likely falls.

   - **Cons**:
     - **Increased Computational Cost**: As you increase the number of iterations, cross-validation becomes computationally more expensive, as you need to train and validate the model multiple times.

2. **Optimal Number of Iterations (k)**:

   - There is typically a trade-off between the number of iterations and computational resources. You want to choose a value for k that balances the need for a reliable estimate with practical constraints.

   - Common choices for k include 5-fold, 10-fold, and occasionally 3-fold cross-validation. These values strike a reasonable balance between estimation accuracy and computational cost.

   - In practice, the choice of k may depend on factors such as the size of your dataset, the complexity of your model, and available computational resources. Smaller datasets might benefit from higher values of k, while larger datasets may be adequately evaluated with lower values of k.

3. **Law of Diminishing Returns**:

   - It's important to note that there is a law of diminishing returns when it comes to increasing the number of iterations. After a certain point, increasing k might not yield substantial improvements in the reliability of the performance estimate, but it will significantly increase the computational cost.

   - In cases where computational resources are limited, it's often better to use a moderate value of k and ensure that you have a sufficiently large and representative dataset.

In summary, increasing the number of iterations in cross-validation generally leads to a more reliable estimate of model performance, up to a point. However, the choice of the optimal number of iterations should consider the balance between estimation accuracy and computational cost, with common values like 5-fold or 10-fold cross-validation being practical choices in many situations.

**4 answer**

Increasing the number of iterations (folds) in cross-validation can help mitigate some of the challenges associated with having a very small training dataset or validation dataset to some extent, but it does not completely solve the fundamental limitations imposed by small dataset sizes. Here's how increasing iterations can help and what its limitations are:

**Pros**:

1. **Better Stability**: With more iterations, cross-validation provides a more stable estimate of model performance. It reduces the impact of randomness in data splits, which can be significant when the dataset is small.

2. **Reduced Bias**: More iterations ensure that the model is evaluated on various subsets of the data, helping to reduce bias in performance estimation. This can be valuable when dealing with limited data.

3. **Improved Confidence Intervals**: Averaging results from more iterations allows for more accurate calculation of confidence intervals, which helps you better understand the range of likely model performance.

**Cons**:

1. **Data Limitations Persist**: Increasing iterations doesn't change the fact that you have a small dataset. The fundamental constraint of having limited data still applies. With a very small dataset, the model may struggle to learn complex patterns, and cross-validation can't create more data.

2. **Risk of Overfitting**: If your dataset is extremely small, increasing iterations can lead to models that are too complex and overfit the training data. In such cases, even if cross-validation provides a more reliable estimate of performance on your limited data, the model's actual ability to generalize to new, unseen data may not improve.

3. **Computational Cost**: As you increase the number of iterations, cross-validation becomes computationally more expensive. This can be a concern if you have limited computational resources.

4. **Diminishing Returns**: There is a point of diminishing returns where increasing the number of iterations doesn't significantly improve the reliability of performance estimation, but it significantly increases computational costs.

In summary, while increasing the number of iterations in cross-validation can provide more stable and less biased estimates of model performance, it cannot fully compensate for the limitations imposed by having a very small training or validation dataset. Small datasets inherently have constraints on model learning and generalization, and additional iterations primarily help in improving the reliability of performance estimation within those constraints. If possible, obtaining more data or using techniques like data augmentation can be more effective in improving model performance with small datasets.